In [1]:
import scipy.stats as stats
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute._iterative import IterativeImputer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import dateutil.parser as parser
from sklearn.impute import SimpleImputer as Imputer, SimpleImputer
from sklearn.neighbors import NearestNeighbors as neigh
from sklearn.neighbors import KNeighborsRegressor as neigh
from sklearn.model_selection import train_test_split as split
from sklearn import metrics
from math import sqrt
import json
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from missingpy import KNNImputer, MissForest
from sklearn.tree import DecisionTreeClassifier
import category_encoders as ce
import copy
from sklearn.tree import export_graphviz
from graphviz import Source
from IPython.display import SVG
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

Tato cast je skopirovane vytvorenie stromu z notebooku 'DecisionTreeClassifier'. Pricom jediny rozdiel je to, ze nacitavame ine subory, ktore sme si dobredu pripravili. V tomto pripade nacitavame data, ktorych chybajuce hodnoty (NaN) boli pri numerickych hodnotach nahradene algoritmom KNN.

In [2]:
merged_data = pd.read_csv('trained_datas_KNN.csv',index_col=0)
# merged_data.info()
merged_data = merged_data.drop(['name', 'address', 'date_of_birth'], axis = 1)
categoric = merged_data.select_dtypes(include = ['object'])

encoder = ce.OneHotEncoder()
encoder.fit(categoric)
transformed = encoder.transform(categoric)

merged_data = merged_data.drop(categoric, axis = 1)
merged_data = merged_data.merge(transformed, left_index=True, right_index=True)
merged_data

,education-num,capital-gain,fnlwgt,hours-per-week,capital-loss,mean_glucose,std_glucose,kurtosis_glucose,skewness_glucose,mean_oxygen,...,native-country_37,native-country_38,native-country_39,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,workclass_7
0,14.0,0.0,338816.0,70.0,0.0,95.351562,41.828816,1.280503,3.351277,14.103679,...,0,0,0,1,0,0,0,0,0,0
1,10.0,0.0,194304.0,55.0,0.0,111.164062,51.802174,0.267679,-0.141940,2.902174,...,0,0,0,0,1,0,0,0,0,0
2,10.0,0.0,200700.0,50.0,0.0,14.460938,29.387737,5.600391,34.223119,57.382107,...,0,0,0,0,1,0,0,0,0,0
3,10.0,0.0,132626.0,6.0,0.0,107.796875,41.562709,0.510415,1.672603,1.910535,...,0,0,0,0,1,0,0,0,0,0
4,10.0,0.0,177974.0,70.0,0.0,111.484375,53.068350,0.459786,-0.116594,1.599498,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,9.0,0.0,245482.0,40.0,0.0,103.578125,45.166638,0.552355,0.777646,21.821070,...,0,0,0,0,1,0,0,0,0,0
3929,13.0,0.0,157612.0,55.0,0.0,27.468750,37.283110,4.430580,20.844460,7.814381,...,0,0,0,0,1,0,0,0,0,0
3930,10.0,0.0,379418.0,40.0,0.0,100.859375,32.731324,0.251239,3.491732,4.968227,...,0,0,0,0,1,0,0,0,0,0
3931,4.0,0.0,200500.0,45.0,0.0,40.398438,33.748458,3.933903,19.294670,96.349498,...,0,0,0,0,1,0,0,0,0,0


In [3]:
cls = DecisionTreeClassifier(criterion = 'entropy')

X = merged_data[merged_data.columns.difference(['class'])]
y = merged_data['class']
cls = cls.fit(X, y)

In [4]:
graph = Source(export_graphviz(cls, 
                               out_file=None,
                               feature_names=X.columns,
                               class_names=['no diabetes', 'yes diabetes'],
                               filled = True))

# display(SVG(graph.pipe(format='svg')))

# from IPython.display import HTML # toto je tu len pre to aby sa mi obrazok zmestil na obrazovku
# style = "<style>svg{width:70% !important;height:70% !important;}</style>"
# HTML(style)

In [5]:
val_merged_data = pd.read_csv('validate_datas_KNN.csv',index_col=0)
# val_merged_data.info()
val_merged_data = val_merged_data.drop(['name', 'address', 'date_of_birth'], axis = 1)
categoric = val_merged_data.select_dtypes(include = ['object'])
transformed = encoder.transform(categoric)

val_merged_data = val_merged_data.drop(categoric, axis = 1)
val_merged_data = val_merged_data.merge(transformed, left_index=True, right_index=True)

In [6]:
val_merged_data = val_merged_data.dropna()
val_merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1310
Columns: 117 entries, education-num to workclass_7
dtypes: float64(15), int64(102)
memory usage: 1.2 MB


In [7]:
prediction = cls.predict(val_merged_data[val_merged_data.columns.difference(['class'])])

In [8]:
print(classification_report(val_merged_data['class'],prediction, digits=3))

              precision    recall  f1-score   support

         0.0      0.970     0.946     0.958       963
         1.0      0.859     0.919     0.888       346

    accuracy                          0.939      1309
   macro avg      0.915     0.933     0.923      1309
weighted avg      0.941     0.939     0.940      1309



Povodne nahradenie chybajucich hodnot pouzivalo pri nahradzovani numerickych hodnot KNN algoritmus a MissForest s transformerom. Dosiahli sme accuracy 93.8% a precision 91.3%, ale ako by povedal Vranic, "ja nie som odbornik".

Pri strategii, kde sme vyuzili iba KNN algoritmus, dosiahol strom accuracy 93.9% a precission 91.5%. Teda mozeme spozorovat minimalne zlepsenie pri tejto strategii.

Tu pouzivame rovnaku cast. Avsak na nahradzovanie chybajucich hodnot sme pouzili algoritmus mean, bez PowerTransformeru.

In [9]:
merged_data = pd.read_csv('trained_datas_mean.csv',index_col=0)
# merged_data.info()
merged_data = merged_data.drop(['name', 'address', 'date_of_birth'], axis = 1)
categoric = merged_data.select_dtypes(include = ['object'])

encoder = ce.OneHotEncoder()
encoder.fit(categoric)
transformed = encoder.transform(categoric)

merged_data = merged_data.drop(categoric, axis = 1)
merged_data = merged_data.merge(transformed, left_index=True, right_index=True)
merged_data

,education-num,capital-gain,fnlwgt,hours-per-week,capital-loss,mean_glucose,std_glucose,kurtosis_glucose,skewness_glucose,mean_oxygen,...,native-country_37,native-country_38,native-country_39,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,workclass_7
0,14.0,0.0,338816.0,70.0,0.0,95.351562,41.828816,1.280503,3.351277,14.103679,...,0,0,0,1,0,0,0,0,0,0
1,10.0,0.0,194304.0,55.0,0.0,111.164062,51.802174,0.267679,-0.141940,2.902174,...,0,0,0,0,1,0,0,0,0,0
2,10.0,0.0,200700.0,50.0,0.0,14.460938,29.387737,0.850184,3.519960,57.382107,...,0,0,0,0,1,0,0,0,0,0
3,10.0,0.0,132626.0,6.0,0.0,107.796875,41.562709,0.510415,1.672603,1.910535,...,0,0,0,0,1,0,0,0,0,0
4,10.0,0.0,177974.0,70.0,0.0,111.484375,53.068350,0.459786,-0.116594,1.599498,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,9.0,0.0,245482.0,40.0,0.0,103.578125,45.166638,0.552355,0.777646,21.821070,...,0,0,0,0,1,0,0,0,0,0
3929,13.0,0.0,157612.0,55.0,0.0,27.468750,37.283110,4.430580,20.844460,7.814381,...,0,0,0,0,1,0,0,0,0,0
3930,10.0,0.0,379418.0,40.0,0.0,100.859375,32.731324,0.251239,3.491732,4.968227,...,0,0,0,0,1,0,0,0,0,0
3931,4.0,0.0,200500.0,45.0,0.0,40.398438,33.748458,3.933903,19.294670,96.349498,...,0,0,0,0,1,0,0,0,0,0


In [10]:
cls = DecisionTreeClassifier(criterion = 'entropy')

X = merged_data[merged_data.columns.difference(['class'])]
y = merged_data['class']
cls = cls.fit(X, y)

In [11]:
graph = Source(export_graphviz(cls, 
                               out_file=None,
                               feature_names=X.columns,
                               class_names=['no diabetes', 'yes diabetes'],
                               filled = True))

# display(SVG(graph.pipe(format='svg')))

# from IPython.display import HTML # toto je tu len pre to aby sa mi obrazok zmestil na obrazovku
# style = "<style>svg{width:70% !important;height:70% !important;}</style>"
# HTML(style)

In [12]:
val_merged_data = pd.read_csv('validate_datas_mean.csv',index_col=0)
# val_merged_data.info()
val_merged_data = val_merged_data.drop(['name', 'address', 'date_of_birth'], axis = 1)
categoric = val_merged_data.select_dtypes(include = ['object'])
transformed = encoder.transform(categoric)

val_merged_data = val_merged_data.drop(categoric, axis = 1)
val_merged_data = val_merged_data.merge(transformed, left_index=True, right_index=True)

val_merged_data = val_merged_data.dropna()
val_merged_data.info()

In [14]:
prediction = cls.predict(val_merged_data[val_merged_data.columns.difference(['class'])])

In [15]:
print(classification_report(val_merged_data['class'],prediction, digits=3))

              precision    recall  f1-score   support

         0.0      0.970     0.958     0.964       963
         1.0      0.888     0.916     0.902       346

    accuracy                          0.947      1309
   macro avg      0.929     0.937     0.933      1309
weighted avg      0.948     0.947     0.948      1309



Oproti povodnej strategii (accuracy 93.8%, precision 91.3% a recall 93.3%) sme dosiahli lepsie vysledky a to pri vsetkych meritkach, accuracy bolo 94.7%, precision 92.9% a recall 93.7%.

Z toho vyplyva, ze jednoduchsia strategia pre nahradzovanie chybajucich hodnot (mean) dosiahla lepsie vysledky ako komplikovanejsie strategie pouzitim algoritmov umelej inteligencie (KNN a MissForest).